<a href="https://colab.research.google.com/github/lonestarchaser/SED/blob/main/Step_2_Polygon_Search_Specific_VIZIER_Catalogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's just go to Vizier Catalog Search Tool and pull the polygon points

This was complicated because Vizier can't do an actual polygon search so you have to do a large enough radius around a point and then filter the results over which ones are in the polygon after you pull from the database.


## First, install Astropy libraries and import packages and subpackages

These aren't normally included in Colab. You can save yourself some time and only have to run this once per time you open this.

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery gala

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.8/910.8 kB 21.6 MB/s eta 0:00:00


## Import subpackages


In [2]:
from google.colab import drive
#import numpy as np

#from io import StringIO
#import requests
import pandas as pd
#import matplotlib.pyplot as plt

#from astroquery.ipac.irsa import Irsa

from astropy.coordinates import SkyCoord
import astropy.units as u
from shapely.geometry import Point, Polygon

from astroquery.vizier import Vizier
Vizier.ROW_LIMIT = -1

## Connect to your Google Drive

In order to save the file to Google Drive, you need to mount you Google Drive to this Google Colab notebook. This will allow you to access your Google Drive files directly from your notebook.

It's going to ask for a lot of permissions.  Just get over it.

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


Now you can list the files in that directory.

You can do this to confirm the file is actually there:

``!ls "/content/drive/My Drive/Colab Notebooks" ``

# Connect to the Server and run a query

Here is the documentation for Vizier: https://media.readthedocs.org/pdf/keflavich-astroquery/latest/keflavich-astroquery.pdf



## Catalog Info

I wish I could say I was a genius but this was a lot of chatGPT and troubleshooting and crying and begging.

```
# Specify the IPHAS catalog (example catalog identifier, update if needed)
catalog = "II/321/iphas2"  # Ensure this is the correct identifier for the IPHAS data

# Specify the ASAS-SN catalog (example catalog identifier, update if needed)
catalog = "J/ApJS/253/7/table2"  # Ensure this is the correct identifier for the ASAS-SN data

# Specify the Pan-STARRS catalog (example catalog identifier, update if needed)
catalog = "II/349/ps1"  # Ensure this is the correct identifier for the Pan-STARRS data

```





In [6]:
# ---------------------------------
# Pick one of the two locations below

# Define the polygon coordinates and the center

polygon_coords = [(209, 55), (207, 55), (207, 50), (209, 50)]
center_ra, center_dec = 208, 52.5  # Approximate center of the polygon
filenameTag = '_MyFunPolygon.tbl'


# Define the radius of the search region
radius_deg = 2.0  # Increase the radius to ensure full coverage of the polygon

# ---------------------------------
# Pick one of the catalogs below

# ##### Specify the Pan-STARRS catalog (example catalog identifier, update if needed)

catalog = "II/349/ps1"  # Ensure this is the correct identifier for the Pan-STARRS data
fileName = '/content/drive/My Drive/Colab Notebooks/panstarrs_' + filenameTag


# ##### Specify the ASAS-SN catalog (example catalog identifier, update if needed)

# Maybe it's this one: J/ApJS/244/36 but prolly not cuz no hits
# Or this one: J/ApJS/253/7/table2   but prolly not cuz no hits
#catalog = "II/366/catalog"
#fileName = '/content/drive/My Drive/Colab Notebooks/asas_sn_' + filenameTag

# ##### Specify the IPHAS catalog (example catalog identifier, update if needed)
#catalog = "II/321/iphas2"  # Ensure this is the correct identifier for the IPHAS data
#fileName = '/content/drive/My Drive/Colab Notebooks/iphas2_' + filenameTag


# Stop messing with stuff here
# ---------------------------------

# Create a Vizier instance.  Set row limit to -1 so you actually get all the answers
v = Vizier(columns=['*'], row_limit=-1)

polygon = Polygon(polygon_coords)

# Create a SkyCoord object for the center
center_coord = SkyCoord(ra=center_ra * u.deg, dec=center_dec * u.deg, frame='icrs')

# Execute the query
result = v.query_region(center_coord, radius=radius_deg * u.deg, catalog=catalog)

# Check if the result is not empty
if result:
    # Convert the first table in the result to a Pandas DataFrame
    df = result[0].to_pandas()

    # Save the DataFrame to a file if you want to
    #df.to_csv('/content/drive/My Drive/Colab Notebooks/appleNotFiltered.tbl', sep='\t', index=False)

    # Print column names to verify RA and Dec columns
    print("Columns in the DataFrame:", df.columns)

    # Filter rows inside the polygon
    def is_in_polygon(row):
        point = Point(row['RAJ2000'], row['DEJ2000'])  # Update column names if different
        return polygon.contains(point)

    filtered_df = df[df.apply(is_in_polygon, axis=1)]

    # Print the number of filtered rows
    print("Number of filtered rows:", len(filtered_df))

    # Save the filtered results to a file
    filtered_df.to_csv(fileName, sep='\t', index=False)
    print("Filtered results saved to " + fileName)
else:
    print("No results found.")

Columns in the DataFrame: Index(['RAJ2000', 'DEJ2000', 'objID', 'f_objID', 'Qual', 'e_RAJ2000',
       'e_DEJ2000', '_tab1_10', 'Ns', 'Nd', 'gmag', 'e_gmag', 'gKmag',
       'e_gKmag', 'gFlags', 'rmag', 'e_rmag', 'rKmag', 'e_rKmag', 'rFlags',
       'imag', 'e_imag', 'iKmag', 'e_iKmag', 'iFlags', 'zmag', 'e_zmag',
       'zKmag', 'e_zKmag', 'zFlags', 'ymag', 'e_ymag', 'yKmag', 'e_yKmag',
       'yFlags'],
      dtype='object')
Number of filtered rows: 78836
Filtered results saved to /content/drive/My Drive/Colab Notebooks/panstarrs__MyFunPolygon.tbl
